#Step1. Whisper-small 기본 성능 평가

In [1]:
!pip install transformers jiwer librosa soundfile accelerate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 59.1 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import os
import json
import glob
import pandas as pd
from jiwer import wer, cer

# childrenvoice 루트 경로 (필요하면 여기만 수정)
ROOT_DIR = "/content/drive/MyDrive/childrenvoice"

# test 데이터 루트 (age4 ~ age12 폴더 안에 source / labelling 이 있다고 가정)
TEST_DIR = os.path.join(ROOT_DIR, "testdata")

# 이번 실험에서 사용할 나이대: 4~12세
TARGET_AGES = list(range(4, 13))  # [4,5,6,7,8,9,10,11,12]

print("ROOT_DIR :", ROOT_DIR)
print("TEST_DIR :", TEST_DIR)
print("TARGET_AGES :", TARGET_AGES)

ROOT_DIR : /content/drive/MyDrive/childrenvoice
TEST_DIR : /content/drive/MyDrive/childrenvoice/testdata
TARGET_AGES : [4, 5, 6, 7, 8, 9, 10, 11, 12]


In [9]:
# 셀 2. testdata에서 (audio_path, text) 수집 + age별 최대 125개 선택
# -----------------------------
def extract_text_from_json(json_data):
    """
    AI-Hub 아동 음성 json 구조 기준:
    - 우선 json_data["Transcription"]["LabelText"] 시도
    - 안 맞으면 재귀로 문자열 모아서 가장 긴 문장을 transcript로 사용
    """
    # 1) 우리가 알고 있는 위치 먼저 시도
    try:
        text = json_data["Transcription"]["LabelText"]
        if isinstance(text, str):
            return text.strip()
    except KeyError:
        pass

    # 2) 다른 구조일 경우: 문자열 전부 모아서 가장 긴 것 사용
    def collect_strings(obj, bucket):
        if isinstance(obj, dict):
            for v in obj.values():
                collect_strings(v, bucket)
        elif isinstance(obj, list):
            for item in obj:
                collect_strings(item, bucket)
        elif isinstance(obj, str):
            s = obj.strip()
            if s:
                bucket.append(s)

    all_strings = []
    collect_strings(json_data, all_strings)

    if not all_strings:
        return None

    all_strings.sort(key=len, reverse=True)
    return all_strings[0]


rows = []
missing_json_count = 0

for age in TARGET_AGES:  # [4, 5, ..., 12]
    age_source_dir    = os.path.join(TEST_DIR, f"age{age}", "source")
    age_labelling_dir = os.path.join(TEST_DIR, f"age{age}", "labelling")

    if not os.path.isdir(age_source_dir):
        print(f"⚠️ source 폴더 없음: {age_source_dir}")
        continue

    # 화자 폴더 순회
    for speaker_id in sorted(os.listdir(age_source_dir)):
        speaker_source_dir    = os.path.join(age_source_dir, speaker_id)
        speaker_labelling_dir = os.path.join(age_labelling_dir, speaker_id)

        if not os.path.isdir(speaker_source_dir):
            continue

        wav_list = sorted(glob.glob(os.path.join(speaker_source_dir, "*.wav")))
        if not wav_list:
            print(f"⚠️ wav 없음: {speaker_source_dir}")
            continue

        for wav_path in wav_list:
            utt_id = os.path.splitext(os.path.basename(wav_path))[0]
            json_path = os.path.join(speaker_labelling_dir, utt_id + ".json")

            if not os.path.exists(json_path):
                missing_json_count += 1
                continue

            # json에서 텍스트 추출
            try:
                with open(json_path, "r", encoding="utf-8") as f:
                    data = json.load(f)
                text = extract_text_from_json(data)
            except Exception as e:
                print("⚠️ json 로드/파싱 실패:", json_path, "| 에러:", e)
                continue

            if not text:
                continue

            rows.append({
                "age": age,
                "speaker_id": speaker_id,
                "utt_id": utt_id,
                "audio_path": wav_path,
                "text": text.strip(),
            })

print("✅ json + wav 둘 다 있는 전체 샘플 수:", len(rows))
print("⚠️ json 파일이 없어서 못 매칭한 wav 개수:", missing_json_count)

# 전체 DataFrame
all_df = pd.DataFrame(rows)
all_df = all_df.sort_values(["age", "speaker_id", "utt_id"]).reset_index(drop=True)

print("\n✅ age별 전체 샘플 개수:")
print(all_df["age"].value_counts().sort_index())

# 🔹 age별로 최대 125개씩만 선택
limited_dfs = []
for age in TARGET_AGES:
    sub = all_df[all_df["age"] == age].copy()
    if sub.empty:
        continue
    sub = sub.sort_values(["speaker_id", "utt_id"])
    sub = sub.head(125)   # 각 나이별 최대 125개
    limited_dfs.append(sub)

test_df = pd.concat(limited_dfs, axis=0).reset_index(drop=True)

print("\n✅ age별 선택된 샘플 개수 (각 나이 최대 125개):")
print(test_df["age"].value_counts().sort_index())

print("\n✅ 최종 사용 샘플 수:", len(test_df))
display(test_df.head())

# csv로 저장 (4~12세, age별 최대 125개씩)
csv_path = os.path.join(ROOT_DIR, "test_age4_12_per_age125_with_text.csv")
test_df.to_csv(csv_path, index=False, encoding="utf-8-sig")
print("✅ 테스트 리스트 CSV 저장 완료:", csv_path)

✅ json + wav 둘 다 있는 전체 샘플 수: 2115
⚠️ json 파일이 없어서 못 매칭한 wav 개수: 0

✅ age별 전체 샘플 개수:
age
4     991
5     132
6     133
7     130
8     136
9     129
10    137
11    128
12    199
Name: count, dtype: int64

✅ age별 선택된 샘플 개수 (각 나이 최대 125개):
age
4     125
5     125
6     125
7     125
8     125
9     125
10    125
11    125
12    125
Name: count, dtype: int64

✅ 최종 사용 샘플 수: 1125


,age,speaker_id,utt_id,audio_path,text
0,4,1079,K00011079-AFG10-L4N1D1-E-K0KK-00107381,/content/drive/MyDrive/childrenvoice/testdata/...,나쁜 먼지가 많아요.
1,4,1079,K00021079-AFG10-L4N1D1-E-K0KK-00107404,/content/drive/MyDrive/childrenvoice/testdata/...,큰 소리로 말해요.
2,4,1079,K00031079-AFG10-L4N1D1-E-K0KK-00107429,/content/drive/MyDrive/childrenvoice/testdata/...,옷이 얼룩덜룩 더럽혀 있어요.
3,4,1079,K00041079-AFG10-L4N1D1-E-K0KK-00107445,/content/drive/MyDrive/childrenvoice/testdata/...,책을 펴요.
4,4,1079,K00051079-AFG10-L4N1D1-E-K0KK-00107462,/content/drive/MyDrive/childrenvoice/testdata/...,네모난 문을 열어요.


✅ 테스트 리스트 CSV 저장 완료: /content/drive/MyDrive/childrenvoice/test_age4_12_per_age125_with_text.csv


In [10]:
# 셀 3. Whisper-small Zero-shot 평가 (WER / CER 계산)
# -----------------------------
import torch
import librosa
from transformers import WhisperProcessor, WhisperForConditionalGeneration

# 방금 만든 CSV 불러오기
df = pd.read_csv(csv_path)

# 공백/결측 라벨 제거
df = df[df["text"].astype(str).str.strip() != ""].reset_index(drop=True)
print("평가에 사용할 최종 샘플 수:", len(df))

model_name = "openai/whisper-small"

# language / task 설정
processor = WhisperProcessor.from_pretrained(model_name, language="ko", task="transcribe")
model = WhisperForConditionalGeneration.from_pretrained(model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

# 디코더 언어 프롬프트 고정 (한국어 + transcribe)
model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(
    language="ko",
    task="transcribe",
)

pred_texts = []
ref_texts = []
results = []

for idx, row in df.iterrows():
    audio_path = row["audio_path"]
    ref_text   = str(row["text"]).strip()
    age        = row["age"]
    speaker_id = row["speaker_id"]
    utt_id     = row["utt_id"]

    # 1) 오디오 로드 (16kHz)
    audio, sr = librosa.load(audio_path, sr=16000)

    # 2) Whisper 입력 특징 변환
    input_features = processor(
        audio,
        sampling_rate=16000,
        return_tensors="pt",
    ).input_features.to(device)

    # 3) 모델 추론
    with torch.no_grad():
        predicted_ids = model.generate(input_features)

    hyp_text = processor.batch_decode(
        predicted_ids,
        skip_special_tokens=True
    )[0].strip()

    ref_texts.append(ref_text)
    pred_texts.append(hyp_text)

    results.append({
        "age": age,
        "speaker_id": speaker_id,
        "utt_id": utt_id,
        "audio_path": audio_path,
        "ref_text": ref_text,
        "hyp_text": hyp_text,
    })

    # 앞부분은 눈으로 비교
    if idx < 10:
        print(f"\n[{idx}] age={age}, speaker={speaker_id}, utt_id={utt_id}")
        print("REF:", ref_text)
        print("HYP:", hyp_text)

# 전체 WER, CER 계산
wer_score = wer(ref_texts, pred_texts)
cer_score = cer(ref_texts, pred_texts)

print("\n✅ Whisper-small Zero-shot 결과 (age 4~12, 각 나이 최대 125개)")
print(f"WER: {wer_score:.4f}")
print(f"CER: {cer_score:.4f}")

# 발화별 결과 저장
results_df = pd.DataFrame(results)
save_compare_path = os.path.join(ROOT_DIR, "whisper_small_zeroshot_age4_12_per_age125_results.csv")
results_df.to_csv(save_compare_path, index=False, encoding="utf-8-sig")

print("\n✅ 발화별 비교 결과 CSV 저장 완료:")
print(save_compare_path)

results_df.head()

평가에 사용할 최종 샘플 수: 1125

[0] age=4, speaker=1079, utt_id=K00011079-AFG10-L4N1D1-E-K0KK-00107381
REF: 나쁜 먼지가 많아요.
HYP: 나빠 만지가 맞아요

[1] age=4, speaker=1079, utt_id=K00021079-AFG10-L4N1D1-E-K0KK-00107404
REF: 큰 소리로 말해요.
HYP: 금톨 위로 말아요

[2] age=4, speaker=1079, utt_id=K00031079-AFG10-L4N1D1-E-K0KK-00107429
REF: 옷이 얼룩덜룩 더럽혀 있어요.
HYP: 혹시 알록달록 덜앞혀있다야

[3] age=4, speaker=1079, utt_id=K00041079-AFG10-L4N1D1-E-K0KK-00107445
REF: 책을 펴요.
HYP: 잘 먹어요

[4] age=4, speaker=1079, utt_id=K00051079-AFG10-L4N1D1-E-K0KK-00107462
REF: 네모난 문을 열어요.
HYP: 잘 모른다.

[5] age=4, speaker=1079, utt_id=K00061079-AFG10-L4N1D1-E-K0KK-00107481
REF: 알아요.
HYP: ああ、よー

[6] age=4, speaker=1079, utt_id=K00071079-AFG10-L4N1D1-E-K0KK-00107523
REF: 공책 두 권을 사요.
HYP: 안녕히 계세요

[7] age=4, speaker=1079, utt_id=K00081079-AFG10-L4N1D1-E-K0KK-01115850
REF: 화가 났어요.
HYP: わぁ!ガナッサーよぉ!

[8] age=4, speaker=1079, utt_id=K00091079-AFG10-L4N1D1-E-K0KK-00107575
REF: 끝이 너무 울퉁불퉁해.
HYP: 끝이 나면 불통 불통해

[9] age=4, speaker=1079, utt_id=K00101079-AFG10-L4N1D1

,age,speaker_id,utt_id,audio_path,ref_text,hyp_text
0,4,1079,K00011079-AFG10-L4N1D1-E-K0KK-00107381,/content/drive/MyDrive/childrenvoice/testdata/...,나쁜 먼지가 많아요.,나빠 만지가 맞아요
1,4,1079,K00021079-AFG10-L4N1D1-E-K0KK-00107404,/content/drive/MyDrive/childrenvoice/testdata/...,큰 소리로 말해요.,금톨 위로 말아요
2,4,1079,K00031079-AFG10-L4N1D1-E-K0KK-00107429,/content/drive/MyDrive/childrenvoice/testdata/...,옷이 얼룩덜룩 더럽혀 있어요.,혹시 알록달록 덜앞혀있다야
3,4,1079,K00041079-AFG10-L4N1D1-E-K0KK-00107445,/content/drive/MyDrive/childrenvoice/testdata/...,책을 펴요.,잘 먹어요
4,4,1079,K00051079-AFG10-L4N1D1-E-K0KK-00107462,/content/drive/MyDrive/childrenvoice/testdata/...,네모난 문을 열어요.,잘 모른다.


In [11]:
from jiwer import wer, cer

age_scores = []

for age, sub in results_df.groupby("age"):
    refs = sub["ref_text"].astype(str).tolist()
    hyps = sub["hyp_text"].astype(str).tolist()

    age_wer = wer(refs, hyps)
    age_cer = cer(refs, hyps)

    age_scores.append({
        "age": age,
        "wer": age_wer,
        "cer": age_cer,
        "n_samples": len(sub),
    })

age_scores_df = pd.DataFrame(age_scores).sort_values("age").reset_index(drop=True)
print(age_scores_df)


   age       wer       cer  n_samples
0    4  1.529605  2.068677        125
1    5  0.917431  1.284779        125
2    6  0.739938  0.430669        125
3    7  0.490937  0.245662        125
4    8  0.374286  0.146952        125
5    9  0.951264  0.621671        125
6   10  0.521277  0.246833        125
7   11  0.276970  0.088144        125
8   12  0.247627  0.079078        125
